# Calculating Routes at Scale using SQL on BigQuery

*Miguel Alvarez*

SDSC21 Workshop - Unlocking Spatial Analytics in the cloud with CARTO

## Context

Computing origin-destination matrices and the corresponding routes is essential for solving key challenges in many sectors such as logistics and transportation, e-commerce and quick commerce, urban mobility/micromobility (car, scooter, and bike sharing), and even for investors in road network infrastructure or insurance companies and public authorities for road safety analysis.

Depending on the use case, one might be interested in the shortest, fastest, safest, or even the least polluting route. In addition, as our economy gets more complex and dynamic, so does the need for solutions that scale.

Here, we present [CARTO's Analytics Toolbox](https://docs.carto.com/analytics-toolbox-bq/overview/getting-started/) functionality for [calculating routes at Scale on BigQuery](https://docs.carto.com/analytics-toolbox-bq/sql-reference/routing/). In particular, we will analyze how people in NYC moved on Thanksgiving 2015.


## What we will do

In this workshop, we will walk you through all the steps required for calculating routes at scale using [CARTO's Analytics Toolbox](https://docs.carto.com/analytics-toolbox-bq/overview/getting-started/) for BigQuery.

After this workshop you will know how to:
 - Visualize trip data using [spatial indices](https://docs.carto.com/analytics-toolbox-bq/overview/spatial-indexes/)
 - Generate a compacted network preserving distances
 - Calculate routes
 - Calculate road segment use frequencies

For showcasing this, we will use the following two datasets which are publicly available in BigQuery:
 - NYC yellow taxi trips from 2015: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
 - OSM road network: `bigquery-public-data.geo_openstreetmap.planet_ways`


## How to get started

Visit [CARTO's documentation](https://docs.carto.com/analytics-toolbox-bq/overview/getting-started/) to get started!

## 0. Setup

In [1]:
from google.cloud import bigquery
import os
import pandas as pd

In [2]:
service_account_file = '/Users/malvarez/cartodb-on-gcp-datascience.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_file
client = bigquery.Client()

## 1. Data sourcing

The first step is to source the required data. For routing, there are mainly two sources of information required. 

- First, we need the **road network** over which we will compute routes and distances. [OpenStreetMap](https://www.openstreetmap.org/#map=12/40.7462/-73.9468) data is a fantastic option for this, as it’s freely available in BigQuery. In particular, we are going to use the `bigquery-public-data:geo_openstreetmap.planet_ways` table where public roads are stored.

- Second, we need a set of point location pairs to use as start and end points to compute distances. In order to have an open dataset, we decided to use NYC yellow taxi trips in 2015 for which there are more than 100M trips. They are publicly available in BigQuery under the name `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`.

### 1.1. Network generation

We will generate the network using the function [\`carto-un\`.routing.GENERATE_NETWORK](https://docs.carto.com/analytics-toolbox-bq/sql-reference/routing/#generate_network). This function generates a network graph from an aggregation of LineStrings and their corresponding speed. The resulting network is based on a compacted geometry of the linestring collection where all nodes with only two links are dropped.

We will select as our study area the five counties of NYC. For the county geometries we use BigQuery public dataset `bigquery-public-data.geo_us_boundaries.counties`.

In addition, we can specify the types of roads we want to consider. In this case, we'll use `motorway`, `trunk`, `primary`, `secondary`, `tertiary`, `residential`, and `road`. We could also add `cycleways` or `footways`, depending on our use case.

It's important to note that the calculated network is a simplification of the original one, where distances are preserved. As we can see in the map below, it drastically simplifies the topology whilst ensuring the total path’s lengths are preserved.

In [3]:
query = '''
DECLARE county_geom GEOGRAPHY;

SET county_geom = (
SELECT ST_UNION_AGG(county_geom) AS county_geom
FROM `bigquery-public-data.geo_us_boundaries.counties`
WHERE county_name in ('New York', 'Richmond', 'Kings', 'Queens', 'Bronx') AND state_fips_code = '36');
CREATE TABLE IF NOT EXISTS `cartobq.demos_sdsc21.routing_nyc_road_network` AS
SELECT
*
FROM
UNNEST((
  SELECT
    `carto-un`.routing.GENERATE_NETWORK(ARRAY_AGG(STRUCT(geometry, 1.)))
  FROM
    `bigquery-public-data.geo_openstreetmap.planet_ways`,
    UNNEST(all_tags) tag
  WHERE
      ST_INTERSECTS(geometry,
      county_geom)
    AND key = 'highway'
    AND value IN UNNEST(['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'residential', 'road']) ));
'''

In [5]:
client.query(query)

In [6]:
query = '''
SELECT * 
FROM `cartobq.demos_sdsc21.routing_nyc_road_network`
LIMIT 10
'''

In [7]:
network = client.query(query).to_dataframe()
network

,src_idx,src_geo,dest_idx,dest_geo,weight
0,15120,POINT(-73.79481 40.72803),15121,POINT(-73.79389 40.72822),79.607033
1,15120,POINT(-73.79481 40.72803),17296,POINT(-73.79498 40.72892),100.141560
2,15120,POINT(-73.79481 40.72803),44642,POINT(-73.79459 40.72689),128.282611
3,15120,POINT(-73.79481 40.72803),3943,POINT(-73.79572 40.72784),79.828638
4,3726,POINT(-74.12401 40.62167),23846,POINT(-74.12077 40.62229),283.016889
5,3726,POINT(-74.12401 40.62167),3725,POINT(-74.12374 40.62097),81.045731
6,3726,POINT(-74.12401 40.62167),3727,POINT(-74.12425 40.62227),69.502831
7,3726,POINT(-74.12401 40.62167),34328,POINT(-74.12478 40.62156),67.753649
8,569,POINT(-73.95325 40.76754),570,POINT(-73.95278 40.76818),81.542318
9,569,POINT(-73.95325 40.76754),1223,POINT(-73.95548 40.76848),215.036670


### 1.1.1. Visualize network

[Link to map](https://gcp-us-east1.app.carto.com/map/4ec3f9d0-95c0-4c9f-b13d-6ac485dc3ae6)

In [ ]:
'''
SELECT ST_MAKELINE(src_geo, dest_geo) as geom FROM `cartobq.demos_sdsc21.routing_nyc_road_network`
'''

<img src="img/nyc_network.png" />

### 1.2. Trip data

#### 1.2.1. Visualize pickup and drop off locations

Before calculating routes, we'd like to visualize where trips start and end. We'll use the H3 [spatial index](https://docs.carto.com/analytics-toolbox-bq/overview/spatial-indexes/) to aggregate trips and understand where most of the trips start and end. Hierarchical spatial indices are an essential tool for analyzing large spatial datasets, especially when dealing with data sources in different spatial aggregations. They provide a direct relationship between grid cells at different resolutions, enabling extremely performant spatial operations.

*Note* we don't need a geometry column to visualize the data as CARTO can work directly with spatial indices.

For this visualization we use the function [\`carto-un\`.h3.LONGLAT_ASH3](https://docs.carto.com/analytics-toolbox-bq/sql-reference/h3/#longlat_ash3) that returns the h3 cell index containing a point with the specified resolution. We're using [h3 resolution](https://h3geo.org/docs/core-library/restable/) 8 that corresponds to cells of size $~0.7373 km^2$.

In [7]:
# Aggregated by pickup location

'''
WITH trips_h3 AS (
    SELECT *, 
        `carto-un`.h3.LONGLAT_ASH3(pickup_longitude, pickup_latitude, 8) AS pickup_h3_id,
        `carto-un`.h3.LONGLAT_ASH3(dropoff_longitude, dropoff_latitude, 8) AS dropoff_h3_id
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE DATE(pickup_datetime)="2015-11-26"
)
SELECT pickup_h3_id, COUNT(vendor_id) AS n_trips
FROM trips_h3 
GROUP BY pickup_h3_id
'''

In [9]:
client.query(query).to_dataframe()

,pickup_h3_id,n_trips
0,882a1008b9fffff,4534
1,882a100f53fffff,1193
2,882a10089bfffff,5529
3,882a100d63fffff,7717
4,882a100d23fffff,6757
...,...,...
703,882a100523fffff,1
704,882a107401fffff,1
705,882a103a6dfffff,1
706,882a100e95fffff,1


In [ ]:
# Aggregated by drop off location

'''
WITH trips_h3 AS (
    SELECT *, 
        `carto-un`.h3.LONGLAT_ASH3(pickup_longitude, pickup_latitude, 8) AS pickup_h3_id,
        `carto-un`.h3.LONGLAT_ASH3(dropoff_longitude, dropoff_latitude, 8) AS dropoff_h3_id
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE DATE(pickup_datetime)="2015-11-26"
)
SELECT dropoff_h3_id, COUNT(vendor_id) AS n_trips
FROM trips_h3 
GROUP BY dropoff_h3_id
'''

<img src="img/nyc_taxi_pu_do.png" />

## 2. Analysis


On the previous map, we can see that cell `882a100f35fffff` has a larger number of trips than its surrounding cells. We are interested in knowing where these people are traveling, the distances traveled, and the main roads used.

### 2.1. Origin-destination matrix calculation

We will calculate the origin-destination matrix and the corresponding routes using the function [\`carto-un\`.routing.FIND_SHORTEST_PATH_FROM_NETWORK](https://docs.carto.com/analytics-toolbox-bq/sql-reference/routing/#find_shortest_path_from_network) which takes a network, a source point, and a destination point as input, and returns the length and the geometry of the shortest path in terms of weights of links between the node closest to the source point and the node closest to the destination point. The algorithm used for calculating the shortest path is the [Dijkstra algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm).

In particular, we calculate the trips starting within cell `882a100f35fffff`.

In [ ]:
# be careful because we need permissions to run this in CARTO workspace
# Cuanto tarda esto? Podemos guardar la tabla o ejecutarlo en vivo o dejar aquí unas vuantas

query = '''
WITH trips_h3 AS (
    SELECT *, 
           ST_GEOGPOINT(pickup_longitude, pickup_latitude) AS pickup_geom,
           `carto-un`.h3.LONGLAT_ASH3(pickup_longitude, pickup_latitude, 8) AS pickup_h3_id,
           ST_GEOGPOINT(dropoff_longitude, dropoff_latitude) AS dropoff_geom,
           `carto-un`.h3.LONGLAT_ASH3(dropoff_longitude, dropoff_latitude, 8) AS dropoff_h3_id
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE DATE(pickup_datetime)="2015-11-26"
),
agg_network AS (
    SELECT ARRAY_AGG(link) as network
    FROM `cartobq.demos_sdsc21.routing_nyc_road_network` link
),
shortest_paths AS (
SELECT `carto-un`.routing.FIND_SHORTEST_PATH_FROM_NETWORK(network,
       pickup_geom,
       dropoff_geom) AS paths, 
       ROW_NUMBER() OVER() AS trip_id 
FROM trips_h3, agg_network
WHERE pickup_h3_id="882a100f35fffff" 
      AND dropoff_h3_id != "882a100f35fffff"
)
SELECT paths.geom, paths.weight, trip_id FROM shortest_paths
'''

In [ ]:
client.query(query)

In [ ]:
.head()

In [10]:
# Compute some statistics on distances

### 2.2. Route visualization

In order to visualize routes, we use four different layers:
- Route linestring geometries generated with the function `carto-un.routing.FIND_SHORTEST_PATH_FROM_NETWORK` in the previous step.
- Pickup locations
- Drop off locations
- H3 cell `882a100f35fffff`

In [ ]:
# If the previous query takes too long, save results on a table and plot the table

In [ ]:
# pickup locations

'''
WITH trips_h3 AS (
    SELECT *, 
           ST_GEOGPOINT(pickup_longitude, pickup_latitude) AS pickup_geom,
           `carto-un`.h3.LONGLAT_ASH3(pickup_longitude, pickup_latitude, 8) AS pickup_h3_id,
           `carto-un`.h3.LONGLAT_ASH3(dropoff_longitude, dropoff_latitude, 8) AS dropoff_h3_id
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE DATE(pickup_datetime)="2015-11-26"
)
SELECT pickup_geom as geom 
FROM trips_h3
WHERE pickup_h3_id="882a100f35fffff" AND dropoff_h3_id!="882a100f35fffff"
'''

In [ ]:
# dropoff locations

'''
WITH trips_h3 AS (
    SELECT *, 
           ST_GEOGPOINT(dropoff_longitude, dropoff_latitude) AS dropoff_geom,
           `carto-un`.h3.LONGLAT_ASH3(pickup_longitude, pickup_latitude, 8) AS pickup_h3_id,
           `carto-un`.h3.LONGLAT_ASH3(dropoff_longitude, dropoff_latitude, 8) AS dropoff_h3_id
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE DATE(pickup_datetime)="2015-11-26"
)
SELECT dropoff_geom as geom 
FROM trips_h3
WHERE pickup_h3_id="882a100f35fffff" AND dropoff_h3_id!="882a100f35fffff"
'''

In [ ]:
# for plotting the origins

'''
SELECT `carto-un`.h3.ST_BOUNDARY("882a100f35fffff") AS geom
'''

<img src="img/nyc_origin_trips.png" />

### 2.3. Road segment frecuency calculation

We are also interested in understanding which roads have the highest use and where they are located. This information od key in many sectors such as logistics for cross decking or fast delivery for bundling deliveries, for example.

We'll start by calculating use frequencies by road segment. The following table takes all routes, split them by road segment, and calculates frequencies.

In [ ]:
query = '''
-- Temporary function to split our routes into segments
-- so we can later analyze their usage independently
CREATE TEMP FUNCTION
 pairwise(myarray ARRAY<STRUCT<idx INT64,
   geom GEOGRAPHY>>) AS ((
   SELECT
     ARRAY_AGG(STRUCT(x AS a,
         myarray[SAFE_OFFSET(off-1)] AS b,
         ST_MAKELINE(x.geom,
           myarray[SAFE_OFFSET(off-1)].geom) AS geom))
   FROM
     UNNEST(myarray) AS x
   WITH OFFSET off
   WHERE myarray[SAFE_OFFSET(off-1)] IS NOT NULL ));

-- Get network to compute the shortest paths
CREATE TABLE IF NOT EXISTS `cartobq.demos_sdsc21.routing_nyc_segment_frequency` AS
WITH agg_network AS (
 SELECT ARRAY_AGG(link) network
 FROM `cartobq.demos_sdsc21.routing_nyc_road_network` link ),

 -- Get taxi trips for 2015-11-26
trips_h3 AS (
    SELECT *, 
           ST_GEOGPOINT(pickup_longitude, pickup_latitude) AS pickup_geom,
           `carto-un`.h3.LONGLAT_ASH3(pickup_longitude, pickup_latitude, 8) AS pickup_h3_id,
           ST_GEOGPOINT(dropoff_longitude, dropoff_latitude) AS dropoff_geom,
           `carto-un`.h3.LONGLAT_ASH3(dropoff_longitude, dropoff_latitude, 8) AS dropoff_h3_id
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE DATE(pickup_datetime)="2015-11-26"
),
trips_h3_filtered AS (
    SELECT * FROM trips_h3
    WHERE pickup_h3_id="882a100f35fffff" AND dropoff_h3_id!="882a100f35fffff"
),

-- Compute shortest paths
 routes AS (
 SELECT `carto-un`.routing.FIND_SHORTEST_PATH_FROM_NETWORK(network,
     pickup_geom,
     dropoff_geom) compacted_route,
   pickup_h3_id, dropoff_h3_id
 FROM
   agg_network, trips_h3_filtered)

-- Get usage frequency for each segment of the resulting routes
SELECT
 ANY_VALUE(segments.geom) geo,
 COUNT(pickup_h3_id) frequency,
FROM routes route,
 UNNEST(pairwise(compacted_route.path)) segments
GROUP BY
 CONCAT(segments.a.idx,segments.b.idx)

'''

In [ ]:
client.query(query)

### 2.4. Road segment frecuency visualization

Finally, we're interesting in visualizing where these road segments with higher frequencies are located.

In [ ]:
''' '''

<img src="img/nyc_segment_frequency.png" />